In [2]:
# 2. .env 파일에서 API 키를 불러옵니다.
import os
from dotenv import load_dotenv

load_dotenv()
print(".env 파일이 로드되었습니다.")

# 3. 전체 시스템의 기억 상자(State)를 정의합니다.
from typing import TypedDict, List, Optional
from langchain_core.documents import Document

class AgentState(TypedDict):
    # --- 공통 입력 ---
    # 사용자의 초기 요청 (채팅 또는 모임 등록 정보)
    # Optional[]은 값이 있을 수도, 없을 수도 있다는 의미입니다.
    user_input: dict
    
    # --- 라우팅 결과 ---
    # 어떤 전문가에게 일을 보낼지 결정
    route: str

    # --- 전문가 에이전트들과 주고받을 데이터 ---
    # 모임 매칭 에이전트용 데이터
    title: Optional[str]
    description: Optional[str]
    
    # 취미 탐색 에이전트용 데이터
    # (설문조사 질문/답변, 최종 추천 결과 등)
    hobby_reco_session: Optional[dict]
    
    # --- 최종 결과 ---
    final_answer: str

print("마스터 에이전트의 AgentState가 정의되었습니다.")

.env 파일이 로드되었습니다.
마스터 에이전트의 AgentState가 정의되었습니다.


In [3]:
# --- 셀 2: 라우터 노드 정의 ---

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. LLM 모델을 준비합니다.
llm = ChatOpenAI(model="gpt-4o-mini")
print("LLM 모델이 준비되었습니다.")

# 2. 라우팅을 위한 프롬프트 템플릿을 만듭니다.
# LLM에게 사용자의 요청을 보여주고, 두 가지 선택지 중 하나를 고르게 합니다.
routing_prompt = ChatPromptTemplate.from_template(
    """당신은 사용자의 요청을 분석하여 어떤 담당자에게 전달해야 할지 결정하는 AI 라우터입니다.
    사용자의 요청을 보고, 아래 두 가지 경로 중 가장 적절한 경로 하나만 골라 그 이름만 정확히 답변해주세요.

    [경로 설명]
    1. `meeting_matching`: 사용자가 '새로운 모임'을 만들려고 할 때, 기존에 있던 '유사한 모임'을 추천해주는 경로입니다. (입력에 title, description 등이 포함됩니다)
    2. `hobby_recommendation`: 사용자에게 '새로운 취미' 자체를 추천해주는 경로입니다. (사용자가 채팅으로 "취미 추천해줘" 와 같이 질문합니다)

    [사용자 요청]:
    {user_input}

    [판단 결과 (meeting_matching 또는 hobby_recommendation)]:
    """
)

# 3. LLM의 답변을 문자열로 받기 위한 체인을 만듭니다.
router_chain = routing_prompt | llm | StrOutputParser()

# 4. 라우터 노드 함수를 정의합니다.
def route_request(state: AgentState):
    """사용자의 입력을 보고 어떤 전문가에게 보낼지 결정합니다."""
    print("--- ROUTING ---")
    
    user_input = state['user_input']
    
    # LLM을 호출하여 라우팅 결정을 내립니다.
    route_decision = router_chain.invoke({"user_input": user_input})
    
    # 불필요한 공백이나 따옴표를 제거합니다.
    cleaned_decision = route_decision.strip().replace("'", "").replace('"', '')
    
    print(f"라우팅 결정: {cleaned_decision}")
    
    # 결정된 경로를 AgentState에 저장합니다.
    return {"route": cleaned_decision}

print("라우터 노드가 정의되었습니다.")

LLM 모델이 준비되었습니다.
라우터 노드가 정의되었습니다.
